<a href="https://colab.research.google.com/github/AshvinVignesh/Final_year/blob/main/final_year_DL_Bilstm_and_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Input , Layer , Bidirectional ,Reshape,MaxPooling2D, Conv1D, MaxPooling1D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
import pandas as pd
# import gensim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report , f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
df = pd.read_csv("Full_data_aspect_removed_words.csv")
df.head()

,original_text,sentences_with_pos_tags,lang,sentiment,clean_text,Acting,direction,Music,VFX,Genre,excitement,ovr_sent,stop_words
0,இரைச்சல் இல்லாத தெளிவான இசை,இரைச்சல்/NOUN இல்லாத/ADJ தெளிவான/ADJ இசை/NOUN,Tamil,POSITIVE,இரைச்சல் இல்லாத தெளிவான இசை,0,0,1,0,0,0,1,"இல்லாத, தெளிவான"
1,harris romba naal kalichu super song,harris/PROPN romba/ADV naal/NOUN kalichu/VERB ...,Code-Mixed,POSITIVE,harris romba naal kalichu super song,0,0,1,0,0,0,1,"romba, naal, kalichu"
2,hukum pondra iraichal padalai ketu kaathu vali...,hukum/PROPN pondra/ADP iraichal/NOUN padalai/N...,Code-Mixed,NEUTRAL,hukum pondra iraichal padalai ketu kaathu vali...,0,0,1,0,0,0,1,"pondr, iraichal, ketu, kaathu, valichuruchu, a..."
3,youtube la kuda editing nalla pannuvanga,youtube/PROPN la/ADP kuda/ADV editing/NOUN nal...,Code-Mixed,NEUTRAL,youtube kuda editing nalla pannuvanga,0,0,0,1,0,0,1,"la, kuda"
4,amaran is also a diwali release,amaran/PROPN is/AUX also/ADV a/DET diwali/PROP...,Code-Mixed,NEUTRAL,amaran diwali release,0,0,0,0,0,0,0,"is, also, a"


In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
X,y = prepare_data(df,'Acting')

In [ ]:
embeddings_index = {}
f = open('/content/ft_300_fyp_150epoch_window2.txt')
for line in f:
    values = line.split()
    word = values[0] # Since the word is at the 0th index
    coefs = np.asarray(values[1:]) # From the 1st position onwards the values are the embedding values
    embeddings_index[word] = coefs
f.close()

In [ ]:
EMBEDDING_DIM =300

# words not found in embedding index will be all-zeros.

In [ ]:
def analyze_all_aspects_dl(df, aspects, sampling_techniques):
    results = {}
    for aspect in aspects:
        print(f"\nAnalyzing aspect: {aspect}")
        X, y = prepare_data(df, aspect)
        aspect_results = {}
        for technique in sampling_techniques:
            print(f"\nUsing {technique} sampling:")
            mean_score, std_score, reports = train_evaluate_dl_model(X, y, technique)
            aspect_results[technique] = {
                'mean_accuracy': mean_score,
                'std_accuracy': std_score,
                'classification_reports': reports
            }
            print(f"Mean Accuracy: {mean_score:.4f} (\u00b1{std_score:.4f})")
        results[aspect] = aspect_results
    return results

In [ ]:
def train_evaluate_dl_model(X, y, sampling_strategy='none'):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    reports = []

    # Tokenize and pad sequences
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X['clean_text'])
    sequences = tokenizer.texts_to_sequences(X['clean_text'])
    padded_sequences = pad_sequences(sequences, maxlen=20)
    word_index = tokenizer.word_index
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Check if the embedding vector has the expected dimension
            if len(embedding_vector) == EMBEDDING_DIM:
                embedding_matrix[i] = embedding_vector
            else:
                print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    target_names = [str(class_label) for class_label in le.classes_]
    # Prepare embedding matrix (assuming it's preloaded as `embedding_matrix`)
    vocab_size = len(word_index) + 1

    for fold, (train_idx, val_idx) in enumerate(skf.split(padded_sequences, y_encoded)):
        X_train, X_val = padded_sequences[train_idx], padded_sequences[val_idx]
        y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]

        # Apply undersampling if required
        if sampling_strategy == 'undersample':
            sampler = RandomUnderSampler(random_state=42)
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_train_flat, y_train = sampler.fit_resample(X_train_flat, y_train)
            X_train = X_train_flat.reshape(-1, 20)  # Reshape back
        elif sampling_strategy == 'oversample':
            sampler = SMOTE(random_state=42)
            # Flatten `X_train` for SMOTE
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_train_flat, y_train = sampler.fit_resample(X_train_flat, y_train)
            # Reshape back to the original dimensions
            X_train = X_train_flat.reshape(-1, 20)

        # Define the model
        input_layer = tf.keras.layers.Input(shape=(20,))
        embedding_layer = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=20, trainable=False)(input_layer)
        # conv = tf.keras.layers.Conv1D(200, kernel_size=3, activation="relu")(embedding_layer)
        bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(embedding_layer)
        drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
        dense = tf.keras.layers.Dense(128, activation='relu')(drop)
        flat = tf.keras.layers.Flatten()(dense)
        output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

        model = tf.keras.Model(input_layer, output_layer)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit(
            X_train, y_train,
            epochs=5,  # You can adjust the number of epochs
            batch_size=32,
            verbose=0
        )

        # Evaluate the model
        y_pred = model.predict(X_val, verbose=0).argmax(axis=1)
        score = accuracy_score(y_val, y_pred)
        scores.append(score)

        # Generate classification report
        report = classification_report(y_val, y_pred, target_names=target_names, output_dict=True)
        reports.append(report)

        print(f"\nFold {fold + 1} Results:")
        print(classification_report(y_val, y_pred, target_names=target_names))

    return np.mean(scores), np.std(scores), reports

own embedding word 2 vec

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.05      0.62      0.09        34
           0       0.71      0.37      0.49       770
           1       0.20      0.19      0.20       221

    accuracy                           0.34      1025
   macro avg       0.32      0.39      0.26      1025
weighted avg       0.58      0.34      0.41      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.07      0.55      0.12        33
           0       0.75      0.36      0.48       769
           1       0.24      0.41      0.30       222

    accuracy                           0.38      1024
   macro avg       0.35      0.44      0.30      1024
weighted avg       0.62      0.38      0.43      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.48      0.53      0.50       343
           0       0.26      0.49      0.34       153
           1       0.66      0.45      0.54       528

    accuracy                           0.48      1024
   macro avg       0.47      0.49      0.46      1024
weighted avg       0.54      0.48      0.50      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.52      0.28      0.37       344
           0       0.25      0.51      0.34       152
           1       0.64      0.63      0.64       528

    accuracy                           0.50      1024
   macro avg       0.47      0.48      0.45      1024
weighted avg       0.54      0.50      0.50      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.50      0.44      0.47       344
           0       0.25      0.46      0.32       152
           1       0.62      0.52      0.57       528

    accuracy                           0.48      1024
   macro avg       0.46      0.47      0.45      1024
weighted avg       0.53      0.48      0.50      1024

Mean Accuracy: 0.4620 (±0.0422)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.08      0.35      0.13        34
           0       0.83      0.54      0.66       770
           1       0.31      0.52      0.39       221

    accuracy                           0.53      1025
   macro avg       0.41      0.47      0.39      1025
weighted avg       0.70      0.53      0.58      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.07      0.45      0.12        33
           0       0.80      0.53      0.64       769
           1       0.32      0.43      0.37       222

    accuracy                           0.51      1024
   macro avg       0.40      0.47      0.38      1024
weighted avg       0.67      0.51      0.56      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.50      0.49      0.50       343
           0       0.26      0.51      0.35       153
           1       0.69      0.51      0.59       528

    accuracy                           0.50      1024
   macro avg       0.49      0.50      0.48      1024
weighted avg       0.56      0.50      0.52      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.47      0.50      0.49       344
           0       0.26      0.41      0.32       152
           1       0.70      0.56      0.62       528

    accuracy                           0.52      1024
   macro avg       0.48      0.49      0.48      1024
weighted avg       0.56      0.52      0.53      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.51      0.56      0.53       344
           0       0.22      0.51      0.30       152
           1       0.71      0.40      0.51       528

    accuracy                           0.47      1024
   macro avg       0.48      0.49      0.45      1024
weighted avg       0.57      0.47      0.49      1024

Mean Accuracy: 0.4876 (±0.0202)


Dravidian embedding word 2 vec

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.07      0.35      0.12        34
           0       0.93      0.42      0.58       770
           1       0.38      0.88      0.53       221

    accuracy                           0.52      1025
   macro avg       0.46      0.55      0.41      1025
weighted avg       0.79      0.52      0.55      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.10      0.67      0.18        33
           0       0.93      0.52      0.67       769
           1       0.48      0.81      0.60       222

    accuracy                           0.59      1024
   macro avg       0.50      0.67      0.48      1024
weighted avg       0.80      0.59      0.64      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.66      0.78      0.71       343
           0       0.33      0.54      0.41       153
           1       0.90      0.63      0.74       528

    accuracy                           0.67      1024
   macro avg       0.63      0.65      0.62      1024
weighted avg       0.74      0.67      0.68      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.66      0.62      0.64       344
           0       0.31      0.63      0.42       152
           1       0.86      0.64      0.73       528

    accuracy                           0.63      1024
   macro avg       0.61      0.63      0.60      1024
weighted avg       0.71      0.63      0.66      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.72      0.67      0.69       344
           0       0.30      0.57      0.40       152
           1       0.84      0.67      0.75       528

    accuracy                           0.65      1024
   macro avg       0.62      0.64      0.61      1024
weighted avg       0.72      0.65      0.68      1024

Mean Accuracy: 0.6516 (±0.0172)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.09      0.12      0.10        34
           0       0.90      0.86      0.88       770
           1       0.62      0.68      0.65       221

    accuracy                           0.79      1025
   macro avg       0.53      0.55      0.54      1025
weighted avg       0.81      0.79      0.80      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.10      0.15      0.12        33
           0       0.87      0.85      0.86       769
           1       0.59      0.60      0.60       222

    accuracy                           0.77      1024
   macro avg       0.52      0.53      0.52      1024
weighted avg       0.79      0.77      0.78      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.72      0.68      0.70       343
           0       0.37      0.38      0.38       153
           1       0.79      0.82      0.81       528

    accuracy                           0.71      1024
   macro avg       0.63      0.63      0.63      1024
weighted avg       0.71      0.71      0.71      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.58      0.76      0.65       344
           0       0.32      0.43      0.36       152
           1       0.88      0.61      0.72       528

    accuracy                           0.63      1024
   macro avg       0.59      0.60      0.58      1024
weighted avg       0.69      0.63      0.64      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.70      0.78      0.74       344
           0       0.39      0.32      0.35       152
           1       0.81      0.79      0.80       528

    accuracy                           0.71      1024
   macro avg       0.63      0.63      0.63      1024
weighted avg       0.71      0.71      0.71      1024

Mean Accuracy: 0.6823 (±0.0291)


Dravidian embedding Fast text

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.07      0.74      0.12        34
           0       0.92      0.37      0.53       770
           1       0.49      0.72      0.58       221

    accuracy                           0.46      1025
   macro avg       0.49      0.61      0.41      1025
weighted avg       0.80      0.46      0.53      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.08      0.70      0.15        33
           0       0.92      0.54      0.68       769
           1       0.52      0.68      0.59       222

    accuracy                           0.58      1024
   macro avg       0.51      0.64      0.47      1024
weighted avg       0.81      0.58      0.65      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.74      0.67      0.70       343
           0       0.33      0.49      0.39       153
           1       0.85      0.78      0.82       528

    accuracy                           0.70      1024
   macro avg       0.64      0.65      0.64      1024
weighted avg       0.74      0.70      0.71      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.65      0.78      0.71       344
           0       0.42      0.36      0.39       152
           1       0.83      0.75      0.79       528

    accuracy                           0.70      1024
   macro avg       0.63      0.63      0.63      1024
weighted avg       0.71      0.70      0.70      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.75      0.57      0.65       344
           0       0.31      0.60      0.41       152
           1       0.83      0.74      0.78       528

    accuracy                           0.66      1024
   macro avg       0.63      0.64      0.61      1024
weighted avg       0.73      0.66      0.68      1024

Mean Accuracy: 0.6788 (±0.0280)


In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.08      0.09      0.08        34
           0       0.89      0.90      0.90       770
           1       0.67      0.66      0.67       221

    accuracy                           0.82      1025
   macro avg       0.55      0.55      0.55      1025
weighted avg       0.82      0.82      0.82      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.10      0.27      0.15        33
           0       0.89      0.77      0.83       769
           1       0.54      0.67      0.60       222

    accuracy                           0.73      1024
   macro avg       0.51      0.57      0.53      1024
weighted avg       0.79      0.73      0.76      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.68      0.78      0.73       343
           0       0.35      0.29      0.32       153
           1       0.83      0.79      0.81       528

    accuracy                           0.72      1024
   macro avg       0.62      0.62      0.62      1024
weighted avg       0.71      0.72      0.71      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.63      0.73      0.68       344
           0       0.34      0.53      0.41       152
           1       0.89      0.66      0.76       528

    accuracy                           0.66      1024
   macro avg       0.62      0.64      0.61      1024
weighted avg       0.72      0.66      0.68      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.79      0.60      0.69       344
           0       0.40      0.47      0.43       152
           1       0.74      0.82      0.78       528

    accuracy                           0.70      1024
   macro avg       0.65      0.63      0.63      1024
weighted avg       0.71      0.70      0.70      1024

Mean Accuracy: 0.6846 (±0.0294)


In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
aspects = ['Acting', 'direction', 'Music','ovr_sent']

In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.7797, Training Accuracy: 0.7798
Fold 2 CV Accuracy: 0.6650, Training Accuracy: 0.7158
Fold 3 CV Accuracy: 0.6345, Training Accuracy: 0.6845
Fold 4 CV Accuracy: 0.7690, Training Accuracy: 0.8218
Fold 5 CV Accuracy: 0.4289, Training Accuracy: 0.4426
Fold 6 CV Accuracy: 0.7030, Training Accuracy: 0.7389
Fold 7 CV Accuracy: 0.5838, Training Accuracy: 0.5957
Fold 8 CV Accuracy: 0.4848, Training Accuracy: 0.5898
Fold 9 CV Accuracy: 0.6574, Training Accuracy: 0.7153
Fold 10 CV Accuracy: 0.7487, Training Accuracy: 0.7902
Average Training Accuracy across folds: 0.6874
Best CV model Training Accuracy on full training set: 0.7798
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.19      0.81      0.30       131
           0       0.90      0.81      0.85      2962
           1       0.79      0.67      0.72       848

    accuracy                           0.78      3941
   macro av

In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['R_clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['R_clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['R_clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['R_clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.7215, Training Accuracy: 0.7628
Fold 2 CV Accuracy: 0.6650, Training Accuracy: 0.7341
Fold 3 CV Accuracy: 0.7690, Training Accuracy: 0.8427
Fold 4 CV Accuracy: 0.7843, Training Accuracy: 0.7798
Fold 5 CV Accuracy: 0.4569, Training Accuracy: 0.5007
Fold 6 CV Accuracy: 0.8046, Training Accuracy: 0.8255
Fold 7 CV Accuracy: 0.5102, Training Accuracy: 0.5145
Fold 8 CV Accuracy: 0.7614, Training Accuracy: 0.8275
Fold 9 CV Accuracy: 0.6650, Training Accuracy: 0.7398
Fold 10 CV Accuracy: 0.5355, Training Accuracy: 0.5898
Average Training Accuracy across folds: 0.7117
Best CV model Training Accuracy on full training set: 0.8234
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.39      0.54      0.45       131
           0       0.90      0.87      0.88      2962
           1       0.67      0.72      0.69       848

    accuracy                           0.82      3941
   macro av

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['R_clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['R_clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['R_clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


Modert Bert contexiualised embedding

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_train_embedding = np.load("/content/drive/MyDrive/rom_contexualised_embedding/Rom_Train_modernbert_token_embeddings.npy")
X_test_embedding = np.load("/content/drive/MyDrive/rom_contexualised_embedding/Rom_Test_modernbert_token_embeddings.npy")

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")

In [ ]:
def train_final_dl_models_from_csv(train_df, test_df, aspects, X_train_embedding, X_test_embedding):
    """
    For each aspect, uses precomputed embedding arrays (X_train_embedding and X_test_embedding)
    as input features. The corresponding target labels are taken from the CSV files.
    Performs 10-fold cross validation on the training data to select the best deep learning model,
    and evaluates the best model on both the training and test sets, printing classification reports.

    Parameters:
        train_df (pd.DataFrame): Training data CSV with target columns (aspects).
        test_df (pd.DataFrame): Test data CSV with target columns.
        aspects (list of str): List of aspect column names to process.
        X_train_embedding (np.array): Precomputed training embeddings of shape (n_train, seq_length, hidden_size).
        X_test_embedding (np.array): Precomputed test embeddings of shape (n_test, seq_length, hidden_size).

    Returns:
        best_models: dict of best models (with their label encoder) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}

    # Use the sequence length and hidden size from the precomputed embeddings.
    maxlen = X_train_embedding.shape[1]       # e.g., number of tokens (predefined in your embedding process)
    hidden_size = X_train_embedding.shape[2]    # e.g., 768 for BERT-base variants

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Extract the target labels for the current aspect from the CSV files.
        # It is assumed that the aspect is a column in your CSV files.
        y_train = train_df[aspect].values
        y_test = test_df[aspect].values

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)
        target_names = [str(label) for label in le.classes_]

        # Use the precomputed embeddings directly as features.
        X_train_padded = X_train_embedding  # Already of shape (n_train, maxlen, hidden_size)
        X_test_padded = X_test_embedding    # Already of shape (n_test, maxlen, hidden_size)

        # Set up cross-validation on the training data.
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalanced classes.
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture using the precomputed embeddings as input.
            input_layer = tf.keras.layers.Input(shape=(maxlen, hidden_size))
            bi_lstm = tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)
            )(input_layer)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train on the current CV fold using class weights.
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold.
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold.
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy.
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the full training set.
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set.
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports

In [ ]:
aspects = ['Acting', 'direction', 'Music','ovr_sent']

In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df, test_df, aspects, X_train_embedding, X_test_embedding)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.0987, Training Accuracy: 0.0953
Fold 2 CV Accuracy: 0.1371, Training Accuracy: 0.1331
Fold 3 CV Accuracy: 0.2030, Training Accuracy: 0.2021
Fold 4 CV Accuracy: 0.3680, Training Accuracy: 0.3589
Fold 5 CV Accuracy: 0.1142, Training Accuracy: 0.1215
Fold 6 CV Accuracy: 0.1168, Training Accuracy: 0.1334
Fold 7 CV Accuracy: 0.1853, Training Accuracy: 0.1432
Fold 8 CV Accuracy: 0.1675, Training Accuracy: 0.1711
Fold 9 CV Accuracy: 0.2081, Training Accuracy: 0.1985
Fold 10 CV Accuracy: 0.1447, Training Accuracy: 0.1336
Average Training Accuracy across folds: 0.1691
Best CV model Training Accuracy on full training set: 0.3598
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.05      0.24      0.08       131
           0       0.77      0.29      0.43      2962
           1       0.24      0.61      0.34       848

    accuracy                           0.36      3941
   macro av

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Mbert contexulised embeddings

In [ ]:
embeddings_index = {}
f = open('/content/drive/My Drive/Mbert_contexualised_embedding/rom_Mbert/mbert_embeddings.txt')
for line in f:
    values = line.split()
    word = values[0] # Since the word is at the 0th index
    coefs = np.asarray(values[1:]) # From the 1st position onwards the values are the embedding values
    embeddings_index[word] = coefs
f.close()

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects , EMBEDDING_DIM):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['R_clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['R_clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['R_clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects,768)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.2937, Training Accuracy: 0.3122
Fold 2 CV Accuracy: 0.4492, Training Accuracy: 0.4638
Fold 3 CV Accuracy: 0.2716, Training Accuracy: 0.2870
Fold 4 CV Accuracy: 0.3350, Training Accuracy: 0.3152
Fold 5 CV Accuracy: 0.4949, Training Accuracy: 0.5072
Fold 6 CV Accuracy: 0.3350, Training Accuracy: 0.3423
Fold 7 CV Accuracy: 0.2919, Training Accuracy: 0.2836
Fold 8 CV Accuracy: 0.3706, Training Accuracy: 0.3910
Fold 9 CV Accuracy: 0.2538, Training Accuracy: 0.2549
Fold 10 CV Accuracy: 0.3883, Training Accuracy: 0.3685
Average Training Accuracy across folds: 0.3526
Best CV model Training Accuracy on full training set: 0.5060
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.07      0.63      0.12       131
           0       0.87      0.47      0.61      2962
           1       0.46      0.60      0.52       848

    accuracy                           0.51      3941
   macro av

In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects,EMBEDDING_DIM ):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(bi_lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports




In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects,768)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.2456, Training Accuracy: 0.2327
Fold 2 CV Accuracy: 0.2183, Training Accuracy: 0.2247
Fold 3 CV Accuracy: 0.6599, Training Accuracy: 0.6465
Fold 4 CV Accuracy: 0.2056, Training Accuracy: 0.2114
Fold 5 CV Accuracy: 0.6320, Training Accuracy: 0.5808
Fold 6 CV Accuracy: 0.1827, Training Accuracy: 0.1816
Fold 7 CV Accuracy: 0.2690, Training Accuracy: 0.2470
Fold 8 CV Accuracy: 0.1827, Training Accuracy: 0.1725
Fold 9 CV Accuracy: 0.6421, Training Accuracy: 0.6256
Fold 10 CV Accuracy: 0.2741, Training Accuracy: 0.2831
Average Training Accuracy across folds: 0.3406


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best CV model Training Accuracy on full training set: 0.6478
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00       131
           0       0.78      0.75      0.76      2962
           1       0.31      0.40      0.35       848

    accuracy                           0.65      3941
   macro avg       0.36      0.38      0.37      3941
weighted avg       0.65      0.65      0.65      3941



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Accuracy: 0.6318
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        34
           0       0.81      0.70      0.75       746
           1       0.30      0.49      0.37       206

    accuracy                           0.63       986
   macro avg       0.37      0.40      0.37       986
weighted avg       0.67      0.63      0.64       986


=== Processing aspect: direction ===
Fold 1 CV Accuracy: 0.3570, Training Accuracy: 0.3601
